In [47]:
import json
import numpy as np
import os
from pprint import pprint

task_dir = '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_'

### RACE: Unfiltered Workers
# prompt_type, task_id = 'question', '1553982706'  # Q-only
# prompt_type, task_id = 'context_question', 1553790696  # TFIDF
# prompt_type, task_id = 'context_question', 1553901953  # FastText
### RACE: Filtered Workers
# prompt_type, task_id = 'question', '1554052233'  # Q-only
# prompt_type, task_id = 'quote and question', 1554006689  # TFIDF(Q+A)
# prompt_type, task_id = 'quote and question', 1554130485  # TFIDF(A)
# prompt_type, task_id = 'quote and question', 1554069931  # Search
prompt_type, task_id = 'quote and question', 1554072277  # SL
# prompt_type, task_id = 'quote and question', 1554132868  # SL-Influence
### RACE Test
# prompt_type, task_id = 'passage and question', 1555823963  # Full Passage
# prompt_type, task_id = 'quotes and question', 1555946909  # FastText
# prompt_type, task_id = 'quotes and question', 1555952058  # Search (6-10 sentence incorrectly placed at end)
dataset = 'race'

### DREAM
# prompt_type, task_id = 'question', 1554582693  # Q-only
# prompt_type, task_id = 'quote and question', 1554596686  # TFIDF-QA
# prompt_type, task_id = 'quote and question', 1554587404  # TFIDF-A
# prompt_type, task_id = 'quote and question', 1554662280  # BoW-A
# prompt_type, task_id = 'quote and question', 1554675304  # Search
# prompt_type, task_id = 'quote and question', 1554685131  # SL
# prompt_type, task_id = 'quote and question', 1554692472  # SL-Sents
# prompt_type, task_id = 'quote and question', 1554729998  # SL-Sents-Influence
# prompt_type, task_id = 'quote and question', 1555333992  # SL-Theory-of-Mind
## All answers at once
# prompt_type, task_id = 'question, answers, and quotes', 1555707929  # TFIDF(O): 64.21%: (Less filter / no feedback)
# prompt_type, task_id = 'question, answers, and quotes', 1555722489  # Search: 65.38%: (Less filter / no feedback)
# prompt_type, task_id = 'question and quotes', 1555789302  # SL: 75.17% (4/5 filter)
# prompt_type, task_id = 'question and quotes', 1555812443  # SL: 79.32% Actually: quotes and question (4/5 filter)
# prompt_type, task_id = 'passage and question', 1555804551  # Full Passage: 92.97%
# prompt_type, task_id = 'quotes and question', 1555823257  # FastText (5/5 filter) (77.33%)
# prompt_type, task_id = 'quotes and question', 1555946647  # Search on RACE (4 sentences incorrectly placed at end) (80.84%)
# dataset = 'dream'


# split = 'middle'
split = None

# Set useful variables
task_dir += str(task_id)
num_options = 3 if dataset == 'dream' else 4
options = ['A', 'B', 'C', 'D'][:num_options]
debate_mode_to_option = {'Ⅰ': 'A', 'Ⅱ': 'B', 'Ⅲ': 'C', 'Ⅳ': 'D', 'ⅰ': 'A', 'ⅱ': 'B', 'ⅲ': 'C', 'ⅳ': 'D', None: None}
question_type_labels = ['a', 'c', 'l', 'm', 's'] if dataset == 'dream' else ['a', 'b', 'c', 'd', 'e']

# Read HIT data
print('# HIT Files:', len(os.listdir(task_dir)))
hit_results = []
num_passed_test = 0
for hit_dir in os.listdir(task_dir):
    if hit_dir.startswith('o_'):
        continue
    num_passed_test += 1
    with open(os.path.join(task_dir, hit_dir, 'custom/data.json'), 'r') as file:
        hit_results.append(json.load(file))
        file.close()
if len(os.listdir(task_dir)) != num_passed_test:
    num_total_tested = len(os.listdir(task_dir)) - num_passed_test
    print('# Passed Test:', num_passed_test, '/', num_total_tested, '=', round((100. * num_passed_test) / num_total_tested, 2), '%')

# HIT Files: 277
# Passed Test: 105 / 172 = 61.05 %


In [48]:
num_valid_hits, num_rejected_hits, num_incomplete_hits = 0, 0, 0
metrics = {}
task_ratings = {i: 0 for i in range(11)}
quote_ratings = []
durations = []
durations_by_worker = []
durations_by_worker_with_min_acc = []
min_acc = 85
accuracy_by_worker = {}
max_response_freqs = []
worker_ids = []
hits_by_qid = {}

for hit_result in hit_results:
    if ((len(hit_result['reject_reasons']) > 0) or
        (len(hit_result['block_reasons']) > 0)):
        num_rejected_hits += 1
        print(hit_result['worker_id'], hit_result['assignment_id'],
              '| reject_reasons:', hit_result['reject_reasons'],
              '| block_reasons:', hit_result['block_reasons'],
              '| bonus_reasons: ' + str(hit_result['bonus_reasons']) if 'bonus_reasons' in hit_result else '')
    elif hit_result['feedback'] is None:
        num_incomplete_hits += 1
        continue
    
    worker_ids.append(hit_result['worker_id'])
#     if hit_result['worker_id'] == 'A1PUHCEBSOWETV':
#         print('*** A1PUHCEBSOWETV ***', hit_result['assignment_id'])
    num_valid_hits += 1
    if (hit_result['task_rating'] is not None) and (hit_result['task_rating'].isdigit()):
        task_ratings[int(hit_result['task_rating'])] += 1
    if (hit_result.get('quote_rating') is not None) and (hit_result['quote_rating'].isdigit()):
        quote_ratings.append(int(hit_result['quote_rating']))
    for qtype, qtype_accuracy in hit_result['accuracy'].items():
        accuracy_by_worker[qtype] = accuracy_by_worker.get(qtype, []) + [qtype_accuracy]
    
    hit_durations = []
    response_option_counts = {option: 0 for option in options}
    responses = []
    for prompt in hit_result['data']:
        qid = prompt['sample']['qid']
        if (split is not None) and (split not in qid):
            continue
        hits_by_qid[qid] = hits_by_qid.get(qid, [])
        hits_by_qid[qid].append(prompt)
        model_stance = debate_mode_to_option[prompt['sample']['debate_mode']]
        answer = prompt['sample']['eval_labels'][0]
        human_correct = (prompt['response'] == answer)
        assert answer in options, 'Answer must be in options.'
        
        # Calculate metrics
        if qid not in metrics:
            metrics[qid] = {
                option: {
                    'num': 0,
                    'num_correct': 0,
                    'num_correct_debate_mode': 0,
                    'num_incorrect_debate_mode': 0,
                    'num_correct_with_correct_debate_mode': 0,
                    'num_correct_with_incorrect_debate_mode': 0,
                    'num_debate_mode_responses': 0,
                }
                for option in ([None] if model_stance is None else options)
            }
            for qtype in question_type_labels:
                metrics[qid][qtype] = {
                    'num': 0,
                    'num_correct': 0,
                }
            metrics[qid]['answer'] = answer
        for qtype in set(''.join(prompt['sample'].get('question_type_labels', []))):
            qtype = qtype.lower()
            metrics[qid][qtype]['num'] += 1
            metrics[qid][qtype]['num_correct'] += human_correct
        prompt_metrics = metrics[qid][model_stance]
        prompt_metrics['num'] += 1
        prompt_metrics['num_correct'] += human_correct
        if model_stance == answer:
            prompt_metrics['num_correct_with_correct_debate_mode'] += human_correct
            prompt_metrics['num_correct_debate_mode'] += 1
        else:
            prompt_metrics['num_correct_with_incorrect_debate_mode'] += human_correct
            prompt_metrics['num_incorrect_debate_mode'] += 1
        prompt_metrics['num_debate_mode_responses'] += (prompt['response'] == model_stance)
        
        hit_durations.append(prompt['duration'] / 1000.)
        response_option_counts[prompt['response']] += 1
        responses.append(prompt['response'])
    duration = np.median(np.array(hit_durations))
    durations_by_worker.append(duration)
    durations += hit_durations
    response_options_array = np.array(list(response_option_counts.values()))
    response_options_array = response_options_array / response_options_array.sum()
    max_response_freq = response_options_array.max()
    max_response_freqs.append(max_response_freq)
    acc = round(100 * hit_result['accuracy'][prompt_type])
    if acc >= min_acc:
        durations_by_worker_with_min_acc.append(duration)
#     print(np.array(hit_durations))
    print('| Time:', round(duration, 1),
          '| Acc:', acc,
          '| Max Freq:', round(100 * max_response_freq, 1),
          '| Rate:', hit_result['task_rating'],
          '| Feedback:', hit_result['feedback'],
          '| Quote Rating:', None if 'quote_rating' not in hit_result else hit_result['quote_rating'], 
          '| Quote Desc:', None if 'quote_description' not in hit_result else hit_result['quote_description'])

quote_ratings = np.array(quote_ratings)
durations = np.array(durations)
durations_by_worker = np.array(durations_by_worker)
durations_by_worker_with_min_acc = np.array(durations_by_worker_with_min_acc)
max_response_freqs = np.array(max_response_freqs)
durations.sort()
durations_by_worker.sort()
durations_by_worker_with_min_acc.sort()
max_response_freqs.sort()
for qtype in accuracy_by_worker:
    accuracy_by_worker[qtype] = np.array(accuracy_by_worker[qtype])
    accuracy_by_worker[qtype].sort()
print('REJECTED:', num_rejected_hits)
print('INCOMPLETE:', num_incomplete_hits)
print('VALID:', num_valid_hits)
print('Median Question Duration:', np.median(durations))
print('Mean Question Duration:', np.mean(durations[int(durations.shape[0] / 10.):int(9. * durations.shape[0] / 10.)]))
print('Median Worker Duration:', np.median(durations_by_worker))
print('Median Worker Accuracy:', np.median(accuracy_by_worker[prompt_type]))
print('Median Max Response Freq:', np.median(max_response_freqs))
print('Quote Rating:',
      '| Mean:', round(quote_ratings.mean(), 2),
      '| Median:', round(np.median(quote_ratings), 2),
      '| Std:', round(np.std(quote_ratings), 2))
# pprint(hit_results[0]['data'][0])
# pprint(hit_results[0])

| Time: 28.9 | Acc: 53 | Max Freq: 26.3 | Rate: 8 | Feedback: No need for improvements. Thanks! | Quote Rating: None | Quote Desc: None
| Time: 10.9 | Acc: 53 | Max Freq: 31.6 | Rate: 3 | Feedback: Too many questions that made no sense  | Quote Rating: None | Quote Desc: None
| Time: 10.3 | Acc: 37 | Max Freq: 36.8 | Rate: 4 | Feedback: All good | Quote Rating: None | Quote Desc: None
| Time: 35.7 | Acc: 42 | Max Freq: 36.8 | Rate: 2 | Feedback: Provide more context in the passages. | Quote Rating: None | Quote Desc: None
| Time: 29.4 | Acc: 58 | Max Freq: 42.1 | Rate: 10 | Feedback: I think everything worked great! | Quote Rating: None | Quote Desc: None
| Time: 32.6 | Acc: 58 | Max Freq: 42.1 | Rate: 10 | Feedback: It's interesting overall. Some questions did not make sense, but I am not sure if that's done on purpose | Quote Rating: None | Quote Desc: None
| Time: 17.8 | Acc: 68 | Max Freq: 31.6 | Rate: 10 | Feedback: N/A | Quote Rating: None | Quote Desc: None
| Time: 21.1 | Acc: 4

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:129: RuntimeWarning: Mean of empty slice.
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [49]:
accuracy_by_qtype = {qtype: [] for qtype in question_type_labels}
accuracy_by_sample = []
accuracy_by_sample_correct_debate_mode = []
accuracy_by_sample_incorrect_debate_mode = []
convinced_freqs = []
convinced_freqs_with_correct_debate_mode = []
convinced_freqs_with_incorrect_debate_mode = []
num_evals_by_sample = []
for qid, qid_metrics in metrics.items():
    answer = metrics[qid]['answer']
    for qid_metric_key, prompt in qid_metrics.items():
        if qid_metric_key in question_type_labels:
            qtype = qid_metric_key
            if qid_metrics[qtype]['num'] > 0:
                accuracy_by_qtype[qtype].append(qid_metrics[qtype]['num_correct'] / qid_metrics[qtype]['num'])
            continue
        if not (qid_metric_key in [None] + options):
            continue
        model_stance = qid_metric_key

        # Q-only stats
        prompt_metrics = metrics[qid][model_stance]
        num_evals_by_sample.append(prompt_metrics['num'])
        accuracy_by_sample.append(prompt_metrics['num_correct'] / prompt_metrics['num'])
        if model_stance is None:
            continue
        
        # Debater stats
#         if 'num_debate_mode_responses' not in prompt_metrics:
#             print(qid_metric_key, prompt_metrics)
        convinced_freq = prompt_metrics['num_debate_mode_responses'] / prompt_metrics['num']
        if model_stance == answer:
            convinced_freqs_with_correct_debate_mode.append(convinced_freq)
            accuracy_by_sample_correct_debate_mode.append(
                prompt_metrics['num_correct_with_correct_debate_mode'] /
                prompt_metrics['num_correct_debate_mode'])
        else:
            convinced_freqs_with_incorrect_debate_mode.append(convinced_freq)
            accuracy_by_sample_incorrect_debate_mode.append(
                prompt_metrics['num_correct_with_incorrect_debate_mode'] /
                prompt_metrics['num_incorrect_debate_mode'])
        convinced_freqs.append(convinced_freq)

accuracy_by_qtype = {qtype: (np.array(accuracy_by_qtype[qtype]).mean(), len(accuracy_by_qtype[qtype])) for qtype in question_type_labels}
worker_ids = set(worker_ids)
        
num_evals_by_sample = np.array(num_evals_by_sample)
print('Evals per sample:', num_evals_by_sample.mean())
print('Fraction insuffient evals:', (num_evals_by_sample < 5).mean())

convinced_freqs = np.array(convinced_freqs)
print('Convinced:', round(100 * convinced_freqs.mean(), 2), '%')
convinced_freqs_with_correct_debate_mode = np.array(convinced_freqs_with_correct_debate_mode)
print('- Correct debater:', round(100 * convinced_freqs_with_correct_debate_mode.mean(), 2), '%')
convinced_freqs_with_incorrect_debate_mode = np.array(convinced_freqs_with_incorrect_debate_mode)
print('- Incorrect debater:', round(100 * convinced_freqs_with_incorrect_debate_mode.mean(), 2), '%')

accuracy_by_sample = np.array(accuracy_by_sample)
print('Accuracy:', round(100 * accuracy_by_sample.mean(), 2), '%')
accuracy_by_sample_correct_debate_mode = np.array(accuracy_by_sample_correct_debate_mode)
print('- Correct debater:', round(100 * accuracy_by_sample_correct_debate_mode.mean(), 2), '%')
accuracy_by_sample_incorrect_debate_mode = np.array(accuracy_by_sample_incorrect_debate_mode)
print('- Incorrect debater:', round(100 * accuracy_by_sample_incorrect_debate_mode.mean(), 2), '%')

num_target_evals = 5
print('Extra Evals:', round(((100. * (num_evals_by_sample - num_target_evals).sum()) / num_evals_by_sample.sum()), 2), '%')
num_evals_by_sample.sort()
print('Evals per sample distribution:', num_evals_by_sample)

print('Accuracy/Num-Samples by Q Type:')
pprint(accuracy_by_qtype)

Evals per sample: 5.049479166666667
Fraction insuffient evals: 0.0
Convinced: 42.46 %
- Correct debater: 75.73 %
- Incorrect debater: 31.37 %
Accuracy: 52.21 %
- Correct debater: 75.73 %
- Incorrect debater: 44.38 %
Extra Evals: 0.98 %
Evals per sample distribution: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: Mean of empty slice.
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [50]:
def nps(task_ratings):
    num_ratings = sum(list(task_ratings.values()))
    if num_ratings == 0:
        return None

    nps_sum_ratings = 0
    sum_ratings = 0
    for score, num_raters in task_ratings.items():
        sum_ratings += num_raters * score
        if score >= 9:
            nps_sum_ratings += num_raters
        elif score <= 6:
            nps_sum_ratings -= num_raters
    return round(100 * (nps_sum_ratings / float(num_ratings)), 2), round((sum_ratings / float(num_ratings)), 2)

print('NPS, Mean:', nps(task_ratings))

NPS, Mean: (31.68, 8.09)


In [51]:
print(durations_by_worker_with_min_acc)
print(np.median(durations_by_worker_with_min_acc))

[]
nan


/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [52]:
durations_by_worker

array([10.31  , 10.89  , 10.929 , 11.819 , 11.913 , 12.7   , 12.7055,
       13.206 , 13.2625, 13.285 , 13.557 , 13.694 , 13.9895, 14.09  ,
       14.359 , 14.469 , 14.915 , 15.0655, 15.3275, 15.337 , 15.553 ,
       15.683 , 15.82  , 16.415 , 16.872 , 17.396 , 17.482 , 17.752 ,
       17.957 , 18.119 , 18.359 , 18.4   , 18.422 , 18.603 , 18.941 ,
       19.2   , 19.465 , 19.539 , 19.559 , 20.185 , 20.276 , 20.785 ,
       21.0585, 21.069 , 21.365 , 21.904 , 22.4895, 22.513 , 22.576 ,
       22.583 , 22.836 , 22.837 , 22.859 , 23.128 , 23.276 , 23.499 ,
       23.64  , 23.851 , 23.932 , 24.193 , 24.218 , 24.378 , 25.116 ,
       25.266 , 25.698 , 26.092 , 26.222 , 26.271 , 26.303 , 26.351 ,
       26.426 , 26.807 , 27.46  , 27.553 , 27.638 , 28.587 , 28.86  ,
       28.911 , 28.996 , 29.2255, 29.317 , 29.375 , 29.434 , 29.868 ,
       30.1175, 30.715 , 31.486 , 32.1985, 32.458 , 32.52  , 32.566 ,
       33.801 , 35.453 , 35.687 , 36.217 , 36.568 , 41.82  , 43.89  ,
       43.942 , 44.9

In [15]:
qid = 'test/high/18692.txt/0'
for hit in hits_by_qid[qid]:
    print(hit['response'])

KeyError: 'test/high/18692.txt/0'

In [36]:
(convinced_freqs - convinced_freqs.mean()) / (convinced_freqs.std() / (convinced_freqs.shape[0] ** .5))

array([-10.65029295,  -8.9226973 ,  32.53959819, -19.28827118,
        -2.01231472,  -8.9226973 , -19.28827118, -19.28827118,
        32.53959819,  -2.01231472,  -8.9226973 ,  32.53959819,
       -19.28827118, -19.28827118,  32.53959819,  15.26364174,
        15.26364174,  -8.9226973 , -19.28827118, -19.28827118,
        32.53959819,  32.53959819,  32.53959819,  32.53959819,
       -19.28827118,   1.44287657,  32.53959819,   6.62566351,
       -19.28827118, -19.28827118,  -8.9226973 ,  22.17402432,
       -19.28827118,   1.44287657, -19.28827118,  15.26364174,
       -19.28827118,  32.53959819, -19.28827118, -19.28827118,
       -19.28827118,  32.53959819, -19.28827118,  -8.9226973 ,
        32.53959819,  22.17402432,  -8.9226973 ,  15.26364174,
        22.17402432, -19.28827118,  -2.01231472, -19.28827118,
       -19.28827118,  -2.01231472,  -8.9226973 ,   1.44287657,
        32.53959819,  22.17402432, -19.28827118, -19.28827118,
       -19.28827118, -19.28827118, -19.28827118,  -8.92

In [28]:
convinced_freqs_influence = convinced_freqs.copy()

In [29]:
convinced_freqs_influence

array([1.        , 0.2       , 0.2       , 1.        , 0.75      ,
       0.2       , 0.4       , 1.        , 0.4       , 1.        ,
       0.6       , 0.8       , 0.        , 0.4       , 0.4       ,
       0.75      , 0.        , 0.        , 1.        , 0.2       ,
       0.        , 0.6       , 0.2       , 0.        , 0.6       ,
       0.4       , 0.4       , 0.25      , 0.5       , 0.2       ,
       0.6       , 0.        , 0.        , 0.4       , 1.        ,
       0.5       , 1.        , 1.        , 1.        , 0.6       ,
       0.8       , 0.        , 1.        , 0.4       , 0.        ,
       0.2       , 0.        , 1.        , 0.        , 0.8       ,
       0.5       , 0.4       , 0.        , 0.25      , 0.8       ,
       0.6       , 0.2       , 1.        , 0.6       , 0.25      ,
       0.        , 0.6       , 1.        , 0.6       , 0.2       ,
       1.        , 0.        , 0.        , 1.        , 0.25      ,
       0.        , 0.        , 0.2       , 0.25      , 0.     

In [38]:
convinced_freqs_bow = convinced_freqs.copy()

In [55]:
f1 = convinced_freqs_sl.copy()
f2 = convinced_freqs_bow.copy()

In [56]:
(f1.mean() - f2.mean()) / ((((f1.std() ** 4) / f1.shape[0]) + ((f2.std() ** 4) / f2.shape[0])) ** .5)

5.174862981608884

In [42]:
from scipy.stats import ttest_ind

In [58]:
ttest_ind(f1, f2, equal_var=False)

Ttest_indResult(statistic=1.9361409357134727, pvalue=0.053218142376891237)

In [54]:
convinced_freqs_sl = convinced_freqs.copy()